CNN的经典用例是执行图像分类，为什么会使用CNN呢，用于计算机视觉处理的图像通常为224x224甚至更大。那么构建一个神经网络来处理224x224彩色图像：包括图像中的3个颜色通道（RGB），即224x224x3=150528个输入权重！这种网络中的典型隐藏层可能有1024个节点，因此我们必须仅为第一层训练150528x1024=15亿个权重。第一层就拥有15亿个权重的神经网络，这几乎是不可能完成训练的。

最重要的是其实我们并不需要那么多的权重，相反我们仅知道像素点其相邻的点才是最有用的。因为图像中的物体是由小的局部特征组成的，如圆形虹膜或一张纸的方角。对于第一个隐藏层中的每个节点来说，查看每个像素似乎是很浪费的！

MNIST数据集中的每个图像都是28x28，其中包含了一个居中的灰度数字。每个图像都是28x28=784维的向量，做为输入数据再增加一些隐藏图层，最后输出10个节点的输出层，每个数字1个。

这样做可以完成任务，因为MNIST数据集包含的都是些居中的小图像，因此我们不会遇到图像太大处理不了等问题。

什么是CNN，它是Convolutional Neural Networks的简写，中文翻译过来叫卷积神经网络，就是使用卷积层的神经网络，即Conv层，它们基于卷积的数学运算。Conv图层由一组过滤器组成，就是二维的数字矩阵。如下图是一个3x3过滤器：

![](.\pic\vertical-sobel1.svg)

我们可以通过将滤波器与输入图像进行卷积来产生输出图像，步骤如下：

* 在某个位置将滤镜覆盖在图像顶部。

* 在过滤器中的值与图像中的相应值之间执行逐元素乘法。

* 所有元素求和，求和的结果就是输出图像中目标像素的输出值。

* 重复所有位置

假如有一个4X4的图像使用3X3的过滤器做卷积生成一个2X2的输出图像。如下显示。

![](.\pic\convolve-output.gif)

比如左上角的值是如何算出的（0X-1）+（50X0)+（0X1）+（0X-2）+(80X0)+(31X2)+(33X-1)+(90X0)+(0X1)=29。最后，我们将结果放在输出图像的目标像素中。由于我们的过滤器覆盖在输入图像的左上角，因此我们的目标像素是输出图像的左上角像素，以此类推其它几个值也是如此计算。

我们所使用的过滤器是垂直 Sobel过滤器，就是图像处理中的边缘检测，所以通过卷积输出图像中的亮像素（具有高值的像素）表示原始图像中存在的强边缘。所以通过使用边缘检测滤波器并可以检查出图像中心附近的两个突出的垂直边缘，说到底就是卷积有助于我们查找特定的图像特征。

前面我们先用了3x3过滤器对4x4的输入图像进行卷积，以产生2x2的输出图像，如果我们希望输出图像的大小与输入图像的大小相同。为此，我们需要在图像周围添加上零，然后再用过滤器卷积，这样输出图像就能和输入图像大小一致了。这称为“相同”填充，因为输入和输出具有相同的尺寸。不使用任何填充，有时也被称为“有效”填充。

如前所述，CNN包括使用一组过滤器将输入图像转换为输出图像的卷积层。卷积层的主要参数是它具有的过滤器数量。

对于我们的MNIST CNN，我们将使用一个带有8个过滤器的卷积层做为我们网络中的初始层。这意味着它会将28x28输入图像转换为26x26x8输出向量，因为没有采用填充所以卷积后会少2层，所以输出通向为26x26x8。

# 卷积层

In [ ]:
import numpy as np


class Conv3x3:
  #定义3x3过滤器的卷积层.

  def __init__(self, num_filters):
    self.num_filters = num_filters

    # 过滤器是3X3的数组，除以9是为了降低初始化数据的大小，太大和太小会导致训练网络无效
    self.filters = np.random.randn(num_filters, 3, 3) / 9

  #一个辅助生成器方法，为我们生成所有有效的3x3图像区域，image是2维的numpy数组
  def iterate_regions(self, image):
    h, w = image.shape

    for i in range(h - 2):
      for j in range(w - 2):
        im_region = image[i:(i + 3), j:(j + 3)]
        yield im_region, i, j

  #对输入图像执行卷积操作，返回输出图像    
  def forward(self, input):
    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_filters))#定义输出并初始化为0

    #im_region 包含图像区域的3X3阵列，self.filters是过滤器，两者相乘再使用numpy的sum方法相加就一个卷积的操作得到我们的卷积结果
    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

    return output

# 池化层

卷积层输出的结果通常会导致临近的像素比较相似，这样在输出的结果中会包含很多冗余的信息，意思就是相邻的是同样的目标而不是新的目标，需要偏移很远才能找到新的目标，对于边缘检测来说临近的是相同的边而不是新的内容。

池化层就可以解决这个问题，它可以减小输入的大小，将输入的值更集中一些。池化也只是做很简单的处理比如取最大值，取最小值，取平局值等。如下图所示对4X4的图像执行最大池化产生2X2的输出。

![](.\pic\pool.gif)

对于我们的MNIST CNN，我们将在初始卷积层之后放置一个池大小为2的最大池化层，这样池化层就会将26x26x8输入转换为13x13x8的输出。

![](.\pic\cnn-dims-2.svg)

这个类的工作原理与Conv3x3类类似。为了从给定的图像区域找到最大值，我们使用了np.amax()， numpy数组的取max的方法。我们设置axis=(0,1)是因为我们只想最大化前两个维度，高度和宽度。

In [ ]:
class MaxPool2:

  def iterate_regions(self, image):

    h, w, _ = image.shape
    new_h = h // 2
    new_w = w // 2   #表示整数除法。

    for i in range(new_h):
      for j in range(new_w):
        im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
        yield im_region, i, j

  def forward(self, input):
    '''
    使用给定的输入执行maxpool层的前向传递。
    返回尺寸为（h / 2，w / 2，num_filters）的3d numpy数组。
    -输入是一个3d numpy数组，其维度为（h，w，num_filters）
    '''
    self.last_input = input

    h, w, num_filters = input.shape
    output = np.zeros((h // 2, w // 2, num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.amax(im_region, axis=(0, 1))

    return output

# softmax

为了完善CNN，我们还需要赋予它实际做出预测的能力。我们将使用Softmax层，通常情况下softmax会被用在网络中的最后一层，用来进行最后的分类和归一化。

我们将使用一个有10个节点的softmax层，每个节点代表一个数字，作为我们CNN的最后一层。层中的每个节点将连接到每个输入。应用softmax变换后，概率最大的节点所表示的数字就是CNN的输出。

![](.\pic\cnn-dims-3.svg)

softmax可以帮助我们量化我们对预测的确定程度，这在训练和评估我们的CNN时很有用。更具体地说，使用softmax让我们使用交叉熵损失，分析每个预测的确定程度。这是我们如何计算交叉熵损失。公式为$L=−ln(p_c)$

softmax实现方式如下：

![](.\pic\softmax.png)

* 把所有的指数求和,这个结果是分母。
* 用每个数字的指数作为它的分子。
* 预测的概率=指数/指数和 结果会在0到1之间。



In [ ]:
class Softmax:

  def __init__(self, input_len, nodes):
    # 我们除以输入长度来减少初始值的范围
    self.weights = np.random.randn(input_len, nodes) / input_len
    self.biases = np.zeros(nodes)

  def forward(self, input):

    input = input.flatten()#返回一个一维的数组。因为我们此时不关心它的形状，只为了方便计算

    input_len, nodes = self.weights.shape

    totals = np.dot(input, self.weights) + self.biases# 加权求和
    exp = np.exp(totals)#求指数
    return exp / np.sum(exp, axis=0) # 指数除以指数和算出概率

执行CNN


In [ ]:
import mnist

train_images = mnist.train_images()[:1000]
train_labels = mnist.train_labels()[:1000]
test_images = mnist.test_images()[:1000]
test_labels = mnist.test_labels()[:1000]

conv = Conv3x3(8)                  # 28x28x1 -> 26x26x8
pool = MaxPool2()                  # 26x26x8 -> 13x13x8
softmax = Softmax(13 * 13 * 8, 10) # 13x13x8 -> 10

def forward(image, label):
  '''
  完成CNN的前向传递，计算准确率和交叉熵损失
  '''
  # 把图像数据从[0, 255] 压缩为 [-0.5, 0.5]这样便于计算，通常都是这么做
  out = conv.forward((image / 255) - 0.5)#卷积
  out = pool.forward(out)#池化
  out = softmax.forward(out)#归一化

  # 计算交叉熵损失和精度. np.log()就是e为底取对数
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

  return out, loss, acc

print('MNIST CNN initialized!')

loss = 0
num_correct = 0
for i, (im, label) in enumerate(zip(test_images, test_labels)):
  # Do a forward pass.
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

  # Print stats every 100 steps.
  if i % 100 == 99:
    print(
      '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
      (i + 1, loss / 100, num_correct)
    )
    loss = 0
    num_correct = 0

cnn的介绍到此结束!在这篇文章中，我们讨论了为什么cnn在某些问题上更有用，比如图像分类。介绍了MNIST手写数字数据集。了解了卷积层，谈到池化层，它可以帮助删除除了最有用的功能之外的所有功能。实现了一个Softmax层，所以我们可以使用交叉熵损失，预测准确率。

前面讲解了如何自己实现简单的卷积网络来预测mnist手写体数字数据集，下面讲解使用现在流行的tensorflow深度学习框架如何来训练mnist。

使用一个简单的CNN网络结构如下，括号里边表示tensor经过本层后的输出shape：

    输入层（28 * 28 * 1）
    卷积层1（28 * 28 * 32）
    pooling层1（14 * 14 * 32）
    卷积层2（14 * 14 * 64）
    pooling层2（7 * 7 * 64）
    全连接层（1 * 1024）
    softmax层（10）

# Tensorflow基础

Tensorflow 是谷歌公司推出的机器学习开源神器，是谷歌基于DistBelief进行研发的第二代人工智能学习系统。DistBelief是谷歌内部开发和使用的机器学习框架，但是它依赖与谷歌内部硬件不能广泛使用。谷歌在此基础上开发了Tensorflow框架。

Tensorflow可以理解为一张计算图中“张量的流动”，其由Tensor和Flow两部分组成。Tensor(张量)代表计算图中的边，Flow(流动)代表计算图中的节点所做的操作而形成的数据流动。

Tensorflow将图的定义和图的运行完全分开。它是一个“符号主义”的库。编程模式分为命令式编程和符号式编程。命令方式变成就是按照我们的命令来执行运算过程。符号式编程是我们定义了一个运算指令，但程序执行到这个运算指令时并没有真正去计算，只有到了最后一行才真正的运算。Tensorflow就是属于第二种。

In [ ]:
import tensorflow as tf
#执行加法操作但是打印并没有输出3，因为它并没有真正执行
t = tf.add(1,2)
print(t)#Tensor("Add_1:0", shape=(), dtype=int32)


Tensorflow 中涉及的运算都放在图中，而图的运行只会发生在会话(Session)中。启动会话后，就可以用数据去填充节点，运行运算；关闭会话后就不能计算了。

In [ ]:
sess = tf.Session()#创建会话
print( sess.run(t))#计算 并输出 3
sess.close()

![](.\pic\tensorflow.png)

上图讲述了Tensorflow的运行原理。首先从输入开始，经过塑性后，进行前向传播运算，Relu层（隐藏层）会有两个参数$W_{h1}$和$b_{h1}$，使用Relu层的激活函数进行非线性计算处理后进入Logit层（输出层），Logit层有两个参数$W_{sm}$和$b_{sm}$用于存储计算的结果。完成计算后使用Softmax方法计算出各类输出结果的概率分布。同时用交叉熵度量源样本概率分布和输出结果概率分布之间的相似性。然后使用这些结果来计算梯度，随后进入SGD训练，也就是反向传播的过程，从上往下计算每一层的参数依次进行更新。

## Tensor

张量是最基本的概念，也是Tensorflow中最主要的数据结构。用于在计算图中进行数据传递，但是创建张量后，不会立即在计算图中增加该张量，而需要给该张量赋值给一个变量或占位符，之后才会将该张量增加到计算图中。张量的数据类型有很多和通常的编程语音一样有浮点数，无符号整形，有符号整形等等。具体的实现方式如下：

In [ ]:
row = 3.0
col = 4.0
zero_ts = tf.zeros([row,col]) #值为0，指定维度的张量
ones_ts = tf.ones([row,col])  #值为1，指定维度的张量
filed_ts = tf.fill([3,4],2.0) #指定填充数值为2.0，指定维度的张量
constant_ts = tf.constant([1,2,3]) #指定已知常数张量
randunif = tf.random_uniform([3,4],minval=0,maxval=2) #创建均匀分布随机数的张量

sess = tf.Session()#创建会话
print( sess.run(randunif))#查看输出填入对应的变量名
sess.close()


## Session

会话是计算图的具体执行者，通常是建立会话，会生成一张空图，在会话中添加张量和节点形成一张图，然后执行。
如下代码就是调用会话的生成函数和关闭函数：
sess = tf.Session()#创建会话

print( sess.run(randunif))

sess.close()

另一种方式是利用上下文管理机制自动释放资源如下：

with tf.Session() as sess：

    sess.run()
    
使用这种方式不需要调用close来释放资源，退出with语句时，会话自动关闭并释放资源

## Variable

变量一般用来表示图中的各个计算参数，包括矩阵和向量等，它在图中有固定的位置，不像普通张量那样可以流动。

In [ ]:
m_var = tf.Variable(zero_ts)#声明变量
init_op = tf.global_variables_initializer()#变量的使用需要进行初始化，否则会报错
sess = tf.Session()#创建会话
sess.run(init_op)
print( sess.run(m_var))#查看输出填入对应的变量名
sess.close()

## Placeholder

占位符用户表示输入输出数据的格式，允许传入指定类型和形状的数据。占位符仅仅声明数据位置，告诉系统这里有一个值，现在还没有具体数值。只有通过会话的feed_dict参数获取数据，在图运行是使用获取的数据进行计算，计算完成后获取的数据就会消失。

In [ ]:
x = tf.placeholder(tf.float32)#声明占位符x
y = tf.placeholder(tf.float32)#声明占位符y
z = tf.add(x,y)
sess = tf.Session()#创建会话
print( sess.run([z],feed_dict={x:[1.0,2.0],y:[1.0,2.0]}))#获取数据进行计算
sess.close()

## Operation

操作是Tensorflow的节点，它的输入和输出都是张量。包括运算操作，矩阵操作，神经网络构建等。
![](.\pic\tfop.png)

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#权值初始化
def weight_value(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.01))

#偏置初始化
def biase_value(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

#卷积操作
def conv2d(Inputs,Weight):
    return tf.nn.conv2d(Inputs,Weight,strides=[1,1,1,1],padding='SAME')

#池化操作
def maxpooling(Inputs):
    return tf.nn.max_pool(Inputs,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义输入数据并预处理数据。通过使用nput_data.read_data_sets方法读取MNIST数据，并分别得到训练集的图片和标记的矩阵
#One-Hot编码是分类变量作为二进制向量的表示。这首先要求将分类值映射到整数值。然后，每个整数值被表示为二进制向量，
#除了整数的索引之外，它都是零值，它被标记为1。和BP章节所讲的使用方式一样。
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)
 
x=tf.placeholder(tf.float32,[None,784])#输入图片数据占位符
y=tf.placeholder(tf.float32,[None,10])#输入标签占位符
keep_prob=tf.placeholder(tf.float32)##定义dropout的占位符keep_conv它表示在一层中有多少比例的神经元被保留下来。
 
#定义了两个卷积层，一个全连接层，一个输出层
#所以
Image=tf.reshape(x,[-1,28,28,1])
#第一个卷积层的定义
conv1_W=weight_value([5,5,1,32])
conv1_b=biase_value([32])
conv1_h=tf.nn.relu(conv2d(Image,conv1_W)+conv1_b)
conv1_out=maxpooling(conv1_h)
#第二个卷积层的定义
conv2_W=weight_value([5,5,32,64])
conv2_b=biase_value([64])
conv2_h=tf.nn.relu(conv2d(conv1_out,conv2_W)+conv2_b)
conv2_out=maxpooling(conv2_h)
#全连接层的定义
fcnn_in=tf.reshape(conv2_out,[-1,49*64])
fcnn1_W=weight_value([49*64,1024])
fcnn1_b=biase_value([1024])
fcnn1_out=tf.nn.relu(tf.matmul(fcnn_in,fcnn1_W)+fcnn1_b)
fcnn1_dropout=tf.nn.dropout(fcnn1_out,keep_prob)
 
fcnn2_W=weight_value([1024,10])
fcnn2_b=biase_value([10])
prediction=tf.nn.softmax(tf.matmul(fcnn1_dropout,fcnn2_W)+fcnn2_b)
#采用交叉熵做目标函数
cross_entropy=-tf.reduce_sum(y*tf.log(prediction))
#精确度计算
num=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accurate=tf.reduce_mean(tf.cast(num,tf.float32))
#训练模型
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
 
sess=tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    batch_x,batch_y=mnist.train.next_batch(100)
    sess.run(train_step,feed_dict={x:batch_x,y:batch_y,keep_prob:0.5})
    if step%50==0:
        #没迭代50步输出在测试机上的精确度
        print(sess.run(accurate,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0}))
sess.close()

# Keras

Keras 是一个模型级（model-level）的库，为开发深度学习模型提供了高层次的构建模块。它不处理张量操作、求微分等低层次的运算。相反，它依赖于一个专门的、高度优化的张量库来完成这些运算，这个张量库就是 Keras 的后端引擎（backend engine）。Keras 没有选择单个张量库并将 Keras 实现与这个库绑定，而是以模块化的方式处理这个问题。因此，几个不同的后端引擎都可以无缝嵌入到 Keras 中。目前，Keras 有三个后端实现：TensorFlow 后端、Theano 后端和微软认知工具包（CNTK，Microsoft cognitive toolkit）后端。未来 Keras 可能会扩展到支持更多的深度学习引擎。
![](.\pic\keras.png)

TensorFlow、CNTK 和 Theano 是当今深度学习的几个主要平台。Theano 由蒙特利尔大学的MILA 实验室开发，TensorFlow 由 Google 发，CNTK 由微软开发。你用 Keras 写的每一段代码都可以在这三个后端上运行，无须任何修改。也就是说，你在开发过程中可以在两个后端之间无缝切换，这通常是很有用的。例如，对于特定任务，某个后端的速度更快，那么我们就可以无缝切换过去。我们推荐使用 TensorFlow 后端作为大部分深度学习任务的默认后端，因为它的应用最广泛，可扩展，而且可用于生产环境。通过 TensorFlow（或 Theano、CNTK），Keras 可以在 CPU 和 GPU 上无缝运行。在 CPU 上运行时，TensorFlow 本身封装了一个低层次的张量运算库，叫作 Eigen；在 GPU 上运行时，TensorFlow封装了一个高度优化的深度学习运算库，叫作 NVIDIA CUDA 深度神经网络库（cuDNN）。

典型的 Keras 工作流程如下：

(1) 定义训练数据：输入张量和目标张量。

(2) 定义层组成的网络（或模型），将输入映射到目标。使用Sequential类

(3) 配置学习过程：选择损失函数、优化器和需要监控的指标。比如categorical_crossentropy损失函数，Adam优化器等

(4) 调用模型的 fit 方法在训练数据上进行迭代。


In [ ]:
import numpy as np
from keras.datasets import mnist# keras内置了mnist数据集
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam

# 载入mnist数据集，需要提前下载mnist.npz文件并放到.keras/datasets目录里，本身keras会自动去下载，但是下载地址需要翻墙，所以需要提前
# 下载好压缩包。
(x_train,y_train),(x_test,y_test) = mnist.load_data()
# (60000,28,28)->(60000,28,28,1)
x_train = x_train.reshape(-1,28,28,1)/255.0
x_test = x_test.reshape(-1,28,28,1)/255.0
# 把训练的标签和测试的标签转换成one hot格式，第一个参数是输入数据，第二个参数为类别个数，我们是10个数字所以是10
y_train = np_utils.to_categorical(y_train,num_classes=10)
y_test = np_utils.to_categorical(y_test,num_classes=10)

# 定义顺序模型 通常使用add方法来堆叠模型
model = Sequential() 

# 第一个卷积层
# input_shape 输入平面
# filters 卷积核/滤波器个数
# kernel_size 卷积窗口大小
# strides 缩小比例因
# padding padding方式 same表示保留边界处的卷积结果，输出的形状保持和输入相同；valid表示只进行有效的卷积，对边界数据不处理。 
# activation 激活函数
model.add(Convolution2D(
    input_shape = (28,28,1),#输入数据是28*28
    filters = 32,
    kernel_size = 5,#卷积窗口大小 5*5
    strides = 1,
    padding = 'same',
    activation = 'relu'
))
# 第一个池化层 使用最大统计量池化 池化后变成14X14特征图
model.add(MaxPooling2D(
    pool_size = 2,#池化窗口大小
    strides = 2,#缩小比例因子，2表示缩小一半
    padding = 'same',#padding方式
))
# 第二个卷积层 64个输入 14X14的特征图
model.add(Convolution2D(
    64,
    5,
    strides=1,
    padding='same',
    activation = 'relu'))
# 第二个池化层 池化后7X7特征图
model.add(MaxPooling2D(2,2,'same'))
# 把第二个池化层的输出扁平化为1维 64*7*7
model.add(Flatten())
# 第一个全连接层
model.add(Dense(1024,activation = 'relu'))
# Dropout正则化在网络中使用是为了减少过拟合(在训练过程中误差低，在测试过程中误差高，这种问题是训练数据少导致，没有更多数据的情况)，在训练过程中随机将该层的一些输出特征舍弃（设置为0），
#即被设置为0的特征所占的比例通常在0.2到0.5之间。
model.add(Dropout(0.5))
# 第二个全连接层
model.add(Dense(10,activation='softmax'))

#Adam优化算法是一种代替随机梯度下降过程的优化算法，高效的计算，内存使用少等优点
# 定义优化器 学习速率为10的负4次方
adam = Adam(lr=1e-4)

# 定义优化器，loss function 交叉熵，训练过程中计算准确率
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

# 训练模型
model.fit(x_train,y_train,batch_size=64,epochs=10)

# 评估模型
loss,accuracy = model.evaluate(x_test,y_test)

print('test loss',loss)
print('test accuracy',accuracy)